In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wfdb
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import StratifiedKFold


In [2]:
def collect_and_label(dataset): 
    

    df = pd.read_csv(dataset)
    alpha = df['scp_codes'].str.split("'").str[1].str[-2:]=='MI'  
    beta = df['scp_codes'].str.split("'").str[1]=='NORM'       
    df = df[alpha|beta]
    df['label'] = df['scp_codes'].str.split("'").str[1]      
      
    inst_c1 = df[df['label'] == 'NORM']
    inst_c1 = inst_c1.sample(n = 1000, random_state = 1)
    
    inst_c2 = df[df['label'] == 'ILMI']
    inst_c2 = inst_c2.sample(n = 393, random_state = 1)
    

    df_new = pd.concat([inst_c1, inst_c2], ignore_index = True)
    df_new = df_new.sample(frac = 1, random_state=42)
    
    return  df_new

In [3]:
def div_and_label(dataframe):  
    
    X = dataframe[['ecg_id', 'filename_hr']]   
    X = X.to_numpy()
    y = dataframe['label']
    encoder = LabelEncoder()
    encoder.fit(y)
    encoded_y = encoder.transform(y)
    
    return (X, encoded_y)

In [4]:
alpha = collect_and_label("C:/Users/Pushpam/Downloads/ptbxl_database.csv")  
gamma0, gamma1 = div_and_label(alpha)
gamma0 = gamma0[:,1]

print(gamma0.shape)
print(gamma1.shape)

(1393,)
(1393,)


In [5]:
print(gamma0)

['records500/01000/01561_hr' 'records500/17000/17484_hr'
 'records500/08000/08857_hr' ... 'records500/03000/03389_hr'
 'records500/07000/07602_hr' 'records500/20000/20800_hr']


In [6]:
gamma1

array([1, 0, 1, ..., 0, 1, 0])

In [7]:
from scipy import signal
import matplotlib.pyplot as plt

In [8]:
import wfdb
import numpy as np
import matplotlib.pyplot as plt
import neurokit2 as nk
from scipy.signal import butter, filtfilt, detrend

directory = 'D:/Internship/MIDataset/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1'

X = []
for itr in range(gamma0.shape[0]):
    record_name = str(gamma0[itr])
    
    signal, meta_val = wfdb.rdsamp(directory + '/' + record_name)
    value = signal.T
    ecg_signals = value

    X.append(detrend(ecg_signals))
    

In [9]:
X = np.array(X)
X.shape

(1393, 12, 5000)

In [10]:
y_data = gamma1;
print(y_data.shape)

(1393,)


In [11]:
# windowing X in xnew
# xnew = np.zeros((5*(X.shape[0]),12,1000))
xold = X
yold = y_data
xnew = []
ynew = []
patient_ids = []
for i in range(X.shape[0]):
    xnew.append(X[i,:,0:1000])
    xnew.append(X[i,:,1000:2000])
    xnew.append(X[i,:,2000:3000])
    xnew.append(X[i,:,3000:4000])
    xnew.append(X[i,:,4000:5000])
    for j in range(5):
        ynew.append(y_data[i])
        patient_ids.append(i)
xnew = np.array(xnew)
y_data = np.array(ynew)

In [12]:
from scipy.signal import coherence
from glob import glob
import scipy.io as sio
import scipy.signal as sig
from scipy.signal import coherence, hilbert 

cross_corr_matrices_list = []
coherence_matrices_list = []
pli_matrices_list = []
plv_matrices_list = []

for patient_data in xnew:
    cross_corr_matrix = np.zeros((12, 12))
    coherence_matrix = np.zeros((12, 12))
    pli_matrix = np.zeros((12, 12))
    plv_matrix = np.zeros((12, 12))

    
    for i in range(12):
        for j in range(i, 12):
            lead_i = patient_data[i]
            lead_j = patient_data[j]
            
            cross_corr = np.corrcoef(lead_i, lead_j)[0, 1]
            
            f, coh = coherence(lead_i, lead_j)  
            coherence_value = np.mean(coh)  # Storing the average coherence value
            
            
            analytic_i = hilbert(lead_i)
            analytic_j = hilbert(lead_j)
            
            phase_i = np.angle(analytic_i)
            phase_j = np.angle(analytic_j)
                    
            phase_diff = phase_i - phase_j
              
            pli = np.abs(np.mean(np.sign(np.sin(phase_diff))))
               
            plv = np.abs(np.mean(np.exp(1j * phase_diff)))

            cross_corr_matrix[i, j] = cross_corr
            cross_corr_matrix[j, i] = cross_corr
        
            coherence_matrix[i, j] = coherence_value
            coherence_matrix[j, i] = coherence_value    
            
            pli_matrix[i, j] = pli
            pli_matrix[j, i] = pli
            
            plv_matrix[i, j] = plv
            plv_matrix[j, i] = plv
    
    
    cross_corr_matrices_list.append(cross_corr_matrix)  
    coherence_matrices_list.append(coherence_matrix)
    pli_matrices_list.append(pli_matrix)
    plv_matrices_list.append(plv_matrix)
    
def create_K(A_i, B_i, C_i, D_i):
    top_row = np.concatenate((A_i, B_i), axis=1)
    bottom_row = np.concatenate((C_i, D_i), axis=1)
    return np.concatenate((top_row, bottom_row), axis=0)

K_matrices=[]
for i in range(xnew.shape[0]):
    K_i = create_K(cross_corr_matrices_list[i], coherence_matrices_list[i], pli_matrices_list[i], plv_matrices_list[i])
    K_matrices.append(K_i)

In [22]:
# Save the matrices to an .npz file
np.savez('matrices_data.npz',
         cross_corr=cross_corr_matrices_list,
         coherence=coherence_matrices_list,
         pli=pli_matrices_list,
         plv=plv_matrices_list)

In [28]:
np.save(r'ynew_ALMI_800+164_kernels5000.npy', y_data)
np.save(r'patient_ids_ILMI_1000+393.npy', patient_ids)

In [ ]:
# To load the matrices back in another script, you can use the following code:
loaded_data = np.load('matrices_data.npz')
cross_corr_loaded = loaded_data['cross_corr']
coherence_loaded = loaded_data['coherence']
pli_loaded = loaded_data['pli']
plv_loaded = loaded_data['plv']

In [14]:
def create_K(A_i, B_i, C_i, D_i):
    top_row = np.concatenate((A_i, B_i), axis=1)
    bottom_row = np.concatenate((C_i, D_i), axis=1)
    return np.concatenate((top_row, bottom_row), axis=0)

K_matrices=[]
for i in range(xnew.shape[0]):
    K_i = create_K(cross_corr_matrices_list[i], coherence_matrices_list[i], pli_matrices_list[i], plv_matrices_list[i])
    K_matrices.append(K_i)

In [16]:
np.array(K_matrices).shape

(6965, 24, 24)

In [27]:
y_data.shape

(6965,)

In [17]:
t = np.array(K_matrices).reshape(xnew.shape[0],24,24,1)
print(t.shape)

(6965, 24, 24, 1)


In [18]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import scale

In [19]:
from tensorflow.keras.layers import Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D, AvgPool2D

In [20]:
patient_ids = np.array(patient_ids)

In [52]:
t = np.array(plv_matrices_list)

In [53]:
t.shape

(6965, 12, 12)

In [55]:
# Group-k-fold

from sklearn.metrics import accuracy_score
from sklearn.model_selection import GroupKFold
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GroupKFold
import tensorflow as tf

gkf = GroupKFold(n_splits=5)

sum_accuracy = 0
sum_sensitivity = 0
sum_specificity = 0

for train_idx, test_idx in gkf.split(t, y_data, groups=patient_ids):
    X_train, X_test = t[train_idx], t[test_idx]
    y_train, y_test = y_data[train_idx], y_data[test_idx]

    K.clear_session()


    model = Sequential()

    model.add(Conv2D(10, (3, 3), input_shape=(12, 12, 1)))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Flatten())

    model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='BinaryCrossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train,
                    batch_size=16,
                    epochs=20,
                    verbose=1, 
                    validation_split=0.25)
    
    predictions = model.predict(X_test)
    y_pred = np.round(predictions).astype(int).transpose()
    y_pred = y_pred[0]

    y_pred_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_pred[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_pred_majority.append(majority_vote)

    y_test_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_test[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_test_majority.append(majority_vote)

    accuracy = accuracy_score(y_test_majority, y_pred_majority)
    sum_accuracy += accuracy
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_test_majority, y_pred_majority).ravel()

    # Calculate sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    sum_sensitivity += sensitivity
    sum_specificity += specificity

    print(confusion_matrix(y_test_majority, y_pred_majority))
    print("Accuracy:", accuracy)
    print("Sensitivity:", sensitivity)
    print("Specificity:", specificity)

average_accuracy = sum_accuracy / 5
average_sensitivity = sum_sensitivity / 5
average_specificity = sum_specificity / 5

print("Average Accuracy:", average_accuracy)
print("Average Sensitivity:", average_sensitivity)
print("Average Specificity:", average_specificity)


Epoch 1/20
262/262 [==============================] - 1s 2ms/step - loss: 0.4032 - accuracy: 0.8192 - val_loss: 0.4621 - val_accuracy: 0.8212
Epoch 2/20
262/262 [==============================] - 0s 1ms/step - loss: 0.3292 - accuracy: 0.8614 - val_loss: 0.4359 - val_accuracy: 0.8342
Epoch 3/20
262/262 [==============================] - 0s 1ms/step - loss: 0.3013 - accuracy: 0.8700 - val_loss: 0.4757 - val_accuracy: 0.7559
Epoch 4/20
262/262 [==============================] - 0s 1ms/step - loss: 0.2846 - accuracy: 0.8844 - val_loss: 0.3974 - val_accuracy: 0.8349
Epoch 5/20
262/262 [==============================] - 0s 1ms/step - loss: 0.2640 - accuracy: 0.8939 - val_loss: 0.3730 - val_accuracy: 0.8464
Epoch 6/20
262/262 [==============================] - 0s 1ms/step - loss: 0.2428 - accuracy: 0.9006 - val_loss: 0.3628 - val_accuracy: 0.8536
Epoch 7/20
262/262 [==============================] - 0s 1ms/step - loss: 0.2289 - accuracy: 0.9088 - val_loss: 0.3452 - val_accuracy: 0.8550
Epoch 

262/262 [==============================] - 0s 1ms/step - loss: 0.1084 - accuracy: 0.9579 - val_loss: 0.5337 - val_accuracy: 0.8212
Epoch 17/20
262/262 [==============================] - 0s 1ms/step - loss: 0.0978 - accuracy: 0.9650 - val_loss: 0.5151 - val_accuracy: 0.8392
Epoch 18/20
262/262 [==============================] - 0s 1ms/step - loss: 0.0851 - accuracy: 0.9713 - val_loss: 0.5740 - val_accuracy: 0.8507
Epoch 19/20
262/262 [==============================] - 0s 1ms/step - loss: 0.0748 - accuracy: 0.9729 - val_loss: 0.5763 - val_accuracy: 0.8457
Epoch 20/20
44/44 [==============================] - 0s 731us/step
[[ 70  15]
 [  8 186]]
Accuracy: 0.9175627240143369
Sensitivity: 0.9587628865979382
Specificity: 0.8235294117647058
Epoch 1/20
262/262 [==============================] - 1s 2ms/step - loss: 0.4058 - accuracy: 0.8127 - val_loss: 0.3942 - val_accuracy: 0.8307
Epoch 2/20
262/262 [==============================] - 0s 1ms/step - loss: 0.3289 - accuracy: 0.8606 - val_loss: 0.3